PySpark:

Stack Exchange provides an anonymized [data dump](https://archive.org/details/stackexchange)

In [2]:
%matplotlib inline
import matplotlib
import seaborn as sns
sns.set()
matplotlib.rcParams['figure.dpi'] = 144

Bad XML:


In [2]:
from pyspark import SparkContext
sc = SparkContext("local[*]", "temp")

In [4]:
import os, time
def localpath(path):
    return 'file://' + os.path.join(os.path.abspath(os.path.curdir), path)

In [5]:
lines = sc.textFile(localpath('spark-stats-data/allPosts/'))

In [6]:
lines.count()

212990

relationship between reputation and how long it took each person to ask their first question:
the difference between (`CreationDate` for the User) and when they asked their first question (`CreationDate` for their first question).

In [129]:
q4_list=pd.read_csv("./q4_solution3/part-00000.csv")[:100]
q4_list['diff']=q4_list['diff'].astype(int)
q4_list=q4_list.values.tolist()

veterans:
were active in a time window between 100 and 150 days after account creation

In [ ]:
from pyspark.sql import SQLContext
from pyspark.sql.functions import udf
from pyspark.sql.types import DoubleType
from pyspark import SparkContext
sc = SparkContext("local[*]", "temp")
sqlContext = SQLContext(sc)
import os, time
def localpath(path):
    return 'file://' + os.path.join(os.path.abspath(os.path.curdir), path)
import re
#regex = re.compile('\s*\<row\s+(\w+)=\"(.*?)\".*(\w+)=\"(\d+)\"')
#if re.match('\s*<row\s*(\w+=".*?"\s*)*/>',line):
valid_stack_posts_lines=sc.textFile(localpath("spark-stack-data/allPosts/"))\
.filter(lambda x: (" />" in x)&("  <row" in x)&(' CreationDate=\"' in x))
#.filter(lambda x: regex.match(x))
valid_stack_users_lines = sc.textFile(localpath("spark-stack-data/allUsers/"))\
.filter(lambda x: (" />" in x)&("  <row" in x)&(' CreationDate=\"' in x)&(' Id=\"' in x))
#.filter(lambda x: regex.match(x))
#part1

import re
import time
import datetime

day=24*60*60
def successMatches2(s):
    IdReg= re.search(' Id=\"(\d+)\" ',s)
    CreationDateReg=re.search(' CreationDate=\"(.*?)\" ',s)
    if IdReg:
        Id=int(IdReg.group(1))
    else:
        Id=0
    if CreationDateReg:
        CreationDate= CreationDateReg.group(1)
        time_stamp=time.mktime(datetime.datetime.strptime(CreationDate, "%Y-%m-%dT%H:%M:%S.%f").timetuple())
    else:
        time_stamp=None
    return Id,time_stamp/day

#successMatches2(valid_stack_posts_lines.take(1)[0])

#q6 dates to df
q6_creationdate=valid_stack_users_lines.map(lambda x: successMatches2(x))#.map(lambda line: (line[0],line[1]))
#cols2= Array("Id", "cdate");
df_q6_cdate= q6_creationdate.toDF().selectExpr("_1 as Id", "_2 as cdate")

#part2
import re
import time
import datetime
#time.mktime(datetime.datetime.strptime(s, "%d/%m/%Y").timetuple())
#dateFormat = new SimpleDateFormat("yyyy-MM-dd'T'HH:mm:ss.SSS") 
def successMatches1(s):

    OwnerUserIdReg= re.search(' OwnerUserId=\"(\d+)\" ',s)
    CreationDateReg=re.search(' CreationDate=\"(.*?)\" ',s)
    AnswerCountReg= re.search(' AnswerCount=\"(\d+)\" ',s)
    ViewCountReg= re.search(' ViewCount=\"(\d+)\" ',s)
    FavoriteCountReg=re.search(' FavoriteCount=\"(.*?)\" ',s)
    ScoreReg=re.search(' Score=\"(.*?)\" ',s)
    if OwnerUserIdReg:
        OwnerUserId=int(OwnerUserIdReg.group(1))
    else:
        OwnerUserId=0
    if FavoriteCountReg:
        FavoriteCount= int(FavoriteCountReg.group(1))
    else:
        FavoriteCount=0
    if AnswerCountReg:
        AnswerCount=int(AnswerCountReg.group(1))
    else:
        AnswerCount=0
    if ViewCountReg:
        ViewCount= int(ViewCountReg.group(1))
    else:
        ViewCount=0
    if ScoreReg:
        Score=int(ScoreReg.group(1))
    else:
        Score=0
    if CreationDateReg:
        CreationDate= CreationDateReg.group(1)
        time_stamp=time.mktime(datetime.datetime.strptime(CreationDate, "%Y-%m-%dT%H:%M:%S.%f").timetuple())
    else:
        time_stamp=0
    
    return OwnerUserId,time_stamp/day,Score,FavoriteCount,ViewCount,AnswerCount
valid_stack_posts_lines_questions = valid_stack_posts_lines\
.filter(lambda x: (' PostTypeId=\"1\"' in x)&(' OwnerUserId=\"' in x))
import timeit

start = timeit.default_timer()

q6_first_qs=valid_stack_posts_lines_questions.map(lambda x: successMatches1(x))\
.map(lambda line: (line[0],(line[1],line[2],line[3],line[4],line[5])))\
.reduceByKey(lambda L, R: (L[0],L[1],L[2],L[3],L[4]) if (L[0] < R[0]) else (R[0],R[1],R[2],R[3],R[4]))\
.map(lambda line: (line[0],line[1][0],line[1][1],line[1][2],line[1][3],line[1][4]))
df_q6_user_info= q6_first_qs.toDF().selectExpr("_1 as Id", "_2 as cdate","_3 as Score","_4 as FavoriteCount","_5 as ViewCount","_6 as AnswerCount")

stop = timeit.default_timer()

print('Time: ', stop - start) 

df_q6_user_info.filter(df_q6_user_info.Id==29).show()

#part3
import re
import time
import datetime
from pyspark.sql.functions import min
from pyspark.sql.types import IntegerType
#time.mktime(datetime.datetime.strptime(s, "%d/%m/%Y").timetuple())
#dateFormat = new SimpleDateFormat("yyyy-MM-dd'T'HH:mm:ss.SSS") 
def successMatches3(s):

    OwnerUserIdReg= re.search(' OwnerUserId=\"(\d+)\" ',s)
    CreationDateReg=re.search(' CreationDate=\"(.*?)\" ',s)
    if OwnerUserIdReg:
        OwnerUserId=int(OwnerUserIdReg.group(1))
    else:
        OwnerUserId=0
    if CreationDateReg:
        CreationDate= CreationDateReg.group(1)
        time_stamp=time.mktime(datetime.datetime.strptime(CreationDate, "%Y-%m-%dT%H:%M:%S.%f").timetuple())
    else:
        time_stamp=0    
    return OwnerUserId,time_stamp/day
import timeit

start = timeit.default_timer()

q6_postdate=valid_stack_posts_lines\
.filter(lambda x: (' OwnerUserId=\"' in x)).map(lambda x: successMatches3(x))

df_q6_pc=q6_postdate.toDF().selectExpr("_1 as Id", "_2 as pcdate")

#people.filter(people.age > 30).join(department, people.deptId == department.id) \
#  .groupBy(department.name, "gender").agg({"salary": "avg", "age": "max"})

#q6_vet_joined=df_q6_pc.join(df_q6_cdate, df_q6_pc.Id == df_q6_cdate.Id)
q6_vet_joined=df_q6_pc.join(df_q6_cdate, ["Id"])

df_q6_users=q6_vet_joined.withColumn("diff", q6_vet_joined.pcdate -q6_vet_joined.cdate).na.drop()
df_q6_users=df_q6_users.filter((df_q6_users.diff > 100) & (df_q6_users.diff < 150))

#df_q6_vets=df_q6_users.groupBy(df_q6_users.Id).agg({"diff": "min"})
df_q6_vets=df_q6_users.groupBy(df_q6_users.Id).agg(min(df_q6_users.diff/100)\
.cast(IntegerType()).alias("vet"))

q6_info=df_q6_user_info.join(df_q6_vets,on=["Id"], how='left_outer').na.fill(0)

#Your statements here

stop = timeit.default_timer()

print('Time: ', stop - start) 

import timeit

start = timeit.default_timer()
q6_info.rdd.map(lambda x: (x[6],(x[2],x[3],x[4],x[5],1)))\
.reduceByKey(lambda L, R: (L[0]+R[0],L[1]+R[1],L[2]+R[2],L[3]+R[3],L[4]+R[4]))\
.mapValues(lambda x: (x[0]/x[4],x[1]/x[4],x[2]/x[4],x[3]/x[4],x[4]))\
.map(lambda line: (line[0],line[1][0],line[1][1],line[1][2],line[1][3],line[1][4])).collect()

stop = timeit.default_timer()

print('Time: ', stop - start) 
